In [2]:
import rasterio

In [1]:
import rasterio
from rasterio.merge import merge
import os
import glob

# Folder where the files are located
folder_path = '../data/Cotton_2021_TOA_10m_filter/'

# Find all .tif files in the folder that match the pattern
file_paths = glob.glob(os.path.join(folder_path, 'Cotton_2021_TOA_10m_*_filter.tif'))

# List to hold open datasets
src_files_to_mosaic = []

# Open all the .tif files and add them to the list
for file_path in file_paths[:4]:
    src = rasterio.open(file_path)
    src_files_to_mosaic.append(src)

# Merge all the datasets
mosaic, out_trans = merge(src_files_to_mosaic)

# Copy metadata from the first source file
out_meta = src_files_to_mosaic[0].meta.copy()

# Update metadata for new dimensions, transform, and count
out_meta.update({
    "driver": "GTiff",
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": out_trans
})

# Define the output path for the merged file
output_path = "../data/Cotton_2021_TOA_10m_filter/Cotton_2021_merged_all.tif"

# Write the mosaic to a new file
with rasterio.open(output_path, "w", **out_meta) as dest:
    dest.write(mosaic)

# Close the opened files
for src in src_files_to_mosaic:
    src.close()

print(f"Merged file written to {output_path}")


Merged file written to ../data/Cotton_2021_TOA_10m_filter/Cotton_2021_merged_all.tif


In [9]:
import re
folder_path = '../data/Cotton_2021_TOA_10m_filter/'

# Find all .tif files in the folder that match the pattern
file_paths = glob.glob(os.path.join(folder_path, 'Cotton_2021_TOA_10m_*_filter.tif'))
file_paths 

NameError: name 'glob' is not defined

In [14]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

# Path to the merged file
output_path = "../data/Cotton_2021_TOA_10m_filter/Cotton_2021_TOA_10m_01_filter.tif"

# Open the merged file
with rasterio.open(output_path) as src:
    # Read the merged dataset
    merged_data = src.read(1)  # Assuming it's single-band data
    

In [16]:
src.crs

CRS.from_epsg(4326)

In [18]:
import rasterio
from rasterio.enums import Resampling
import numpy as np

# Path to the merged file
output_path = "../data/Cotton_2021_TOA_10m_filter/Cotton_2021_TOA_10m_02_filter.tif"
resampled_path = "../data/Cotton_2021_TOA_10m_filter/Cotton_2021_TOA_10m_02_resampled_5km.tif"


# Open the merged file
with rasterio.open(output_path) as src:
    # Calculate new transform (affine matrix for georeferencing)
    transform = src.transform

    new_resolution = transform[0]*500

    new_transform = transform * transform.scale(
        (src.width * src.res[0]) / new_resolution,
        (src.height * src.res[1]) / new_resolution
    )
    
    # Calculate new width and height
    new_width = int(src.width * src.res[0] / new_resolution)
    new_height = int(src.height * src.res[1] / new_resolution)
    
    # Resample data
    resampled_data = src.read(
        out_shape=(src.count, new_height, new_width),
        resampling=Resampling.bilinear
    )

    # Update metadata for the new file
    new_meta = src.meta.copy()
    new_meta.update({
        'driver': 'GTiff',
        'height': new_height,
        'width': new_width,
        'transform': new_transform,
        'crs': src.crs,
        'res': (new_resolution, new_resolution)  # Set the new resolution in meters
    })

    # Write the resampled dataset to a new file
    with rasterio.open(resampled_path, 'w', **new_meta) as dst:
        dst.write(resampled_data)

print(f"Resampled file saved to {resampled_path}")


Resampled file saved to ../data/Cotton_2021_TOA_10m_filter/Cotton_2021_TOA_10m_02_resampled_5km.tif
